In [0]:
#Part 1

In [0]:
#Setting spark#

In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('bdeat2').getOrCreate()
spark.version

In [0]:
#Linking Azure to Databricks#

In [0]:
storage_account_name="utsrushad"
storage_account_access_key="cZ2PnhJh4lHSgBlhLWi8ZejCnG66UbCml5phH6sUxK6nT1rCqeiPCQrAoZeF9nYU7xC5IUlHmmL0+ASttzK0kQ=="
blob_container_name="bdeat2"

In [0]:
dbutils.fs.mount(
  source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
  mount_point = f'/mnt/{blob_container_name}/',
  extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
)

In [0]:
#checking the mounted storage#

In [0]:
dbutils.fs.ls("mnt/bdeat2")

In [0]:
#Loading parquets data from azure in order to count rows#

In [0]:
greendata=spark.read.option("header", True).parquet("mnt/bdeat2/*green*.parquet")


In [0]:
yellowdata=spark.read.option("header", True).parquet("mnt/bdeat2/*yellow*.parquet")

In [0]:
print(greendata.count())

In [0]:
print(yellowdata.count())

In [0]:
#Converting yellow Apr 2022 file to csv#

In [0]:
df_yellow_2022_04=spark.read.option("header", True).parquet("mnt/bdeat2/*yellow*2022*04.parquet")


In [0]:
df_yellow_2022_04.write.csv('mnt/bdeat2/yt.csv')

In [0]:
from pyspark.sql.functions import countDistinct

In [0]:
greendata.printSchema()

In [0]:
yellowdata.printSchema()

In [0]:
greendata.cache()

In [0]:
yellowdata.cache()


In [0]:
greendata.show(1)
yellowdata.show(1)

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType

In [0]:
#Converting every columns datatypes to its proper formats# 

In [0]:
greendata_cleaned=greendata.withColumn("VendorID", F.col("VendorID").astype(IntegerType())).withColumn("lpep_pickup_datetime", F.col("lpep_pickup_datetime").astype(TimestampType())).withColumn("lpep_dropoff_datetime", F.col("lpep_dropoff_datetime").astype(TimestampType())).withColumn("RateCodeID", F.col("RateCodeID").astype(IntegerType())).withColumn("PULocationID", F.col("PULocationID").astype(IntegerType())).withColumn("DOLocationID", F.col("DOLocationID").astype(IntegerType())).withColumn("passenger_count", F.col("passenger_count").astype(IntegerType())).withColumn("trip_distance", F.col("trip_distance").astype(IntegerType())).withColumn("fare_amount", F.col("fare_amount").astype(IntegerType())).withColumn("extra", F.col("extra").astype(IntegerType())).withColumn("mta_tax", F.col("tip_amount").astype(IntegerType())).withColumn("tip_amount", F.col("tip_amount").astype(IntegerType())).withColumn("tolls_amount", F.col("tolls_amount").astype(IntegerType())).withColumn("ehail_fee", F.col("ehail_fee").astype(IntegerType())).withColumn("improvement_surcharge", F.col("improvement_surcharge").astype(IntegerType())).withColumn("total_amount", F.col("total_amount").astype(IntegerType())).withColumn("payment_type", F.col("payment_type").astype(IntegerType())).withColumn("trip_type", F.col("trip_type").astype(IntegerType())).withColumn("congestion_surcharge", F.col("congestion_surcharge").astype(IntegerType()))

In [0]:
greendata_cleaned.printSchema()

In [0]:
yellowdata_cleaned=yellowdata.withColumn("VendorID", F.col("VendorID").astype(IntegerType())).withColumn("tpep_pickup_datetime", F.col("tpep_pickup_datetime").astype(TimestampType())).withColumn("tpep_dropoff_datetime", F.col("tpep_dropoff_datetime").astype(TimestampType())).withColumn("RateCodeID", F.col("RateCodeID").astype(IntegerType())).withColumn("PULocationID", F.col("PULocationID").astype(IntegerType())).withColumn("DOLocationID", F.col("DOLocationID").astype(IntegerType())).withColumn("passenger_count", F.col("passenger_count").astype(IntegerType())).withColumn("trip_distance", F.col("trip_distance").astype(IntegerType())).withColumn("fare_amount", F.col("fare_amount").astype(IntegerType())).withColumn("extra", F.col("extra").astype(IntegerType())).withColumn("mta_tax", F.col("tip_amount").astype(IntegerType())).withColumn("tip_amount", F.col("tip_amount").astype(IntegerType())).withColumn("tolls_amount", F.col("tolls_amount").astype(IntegerType())).withColumn("improvement_surcharge", F.col("improvement_surcharge").astype(IntegerType())).withColumn("total_amount", F.col("total_amount").astype(IntegerType())).withColumn("payment_type", F.col("payment_type").astype(IntegerType())).withColumn("congestion_surcharge", F.col("congestion_surcharge").astype(IntegerType())).withColumn("airport_fee", F.col("airport_fee").astype(IntegerType()))

In [0]:
yellowdata_cleaned.cache()

In [0]:
greendata_cleaned.cache()

In [0]:
yellowdata_cleaned.printSchema()

In [0]:
greendata_cleaned.printSchema()

In [0]:
#Filtering trips finishing before starting time#

In [0]:
greendata_cleaned = greendata_cleaned.filter(greendata_cleaned.lpep_pickup_datetime<greendata_cleaned.lpep_dropoff_datetime)

In [0]:
yellowdata_cleaned = yellowdata_cleaned.filter(yellowdata_cleaned.tpep_pickup_datetime<yellowdata_cleaned.tpep_dropoff_datetime)

In [0]:
#As speed=distance/time. So To calculte speed, we need to first calculate time, creating trip_duration variable#

In [0]:
greendata_cleaned=greendata_cleaned.withColumn("trip_duration", F.col("lpep_dropoff_datetime").cast("long")-F.col("lpep_pickup_datetime").cast("long"))

In [0]:
yellowdata_cleaned=yellowdata_cleaned.withColumn("trip_duration", F.col("tpep_dropoff_datetime").cast("long")-F.col("tpep_pickup_datetime").cast("long"))

In [0]:
yellowdata_cleaned.show(1)

In [0]:
greendata_cleaned.show(1)

In [0]:
greendata_cleaned.cache()
yellowdata_cleaned.cache()

In [0]:
#creating a trip_speed variable#

In [0]:
greendata_cleaned=greendata_cleaned.withColumn("trip_speed", F.col("trip_distance").cast("long")/F.col("trip_duration").cast("long"))

In [0]:
yellowdata_cleaned=yellowdata_cleaned.withColumn("trip_speed", F.col("trip_distance").cast("long")/F.col("trip_duration").cast("long"))

In [0]:
#Getting rid of negative speed#

In [0]:
greendata_cleaned = greendata_cleaned.filter(greendata_cleaned.trip_speed>0)

In [0]:
yellowdata_cleaned = yellowdata_cleaned.filter(yellowdata_cleaned.trip_speed>0)

In [0]:
#Getting rid of trips which have very high speed#

In [0]:
greendata_cleaned.agg(F.percentile_approx("trip_speed", [0.10, 0.5, 0.90])).show(truncate=False)

In [0]:
greendata_cleaned=greendata_cleaned.filter(greendata_cleaned.trip_speed<0.0070)

In [0]:
#0.0070 miles per second is about 25 miles per hour
#NewYork speed limit is 30 miles per hor within cities
#Keeping traffic and signal in account i chose 25 miles per hour as a filter threshold

In [0]:
yellowdata_cleaned.agg(F.percentile_approx("trip_speed", [0.1, 0.5, 0.9])).show(truncate=False)

In [0]:
yellowdata_cleaned=yellowdata_cleaned.filter(yellowdata_cleaned.trip_speed<0.0070)

In [0]:
#cleaning data based on trip duration#

In [0]:
greendata_cleaned.agg(F.percentile_approx("trip_duration", [0.05, 0.5, 0.95])).show(truncate=False)

In [0]:
greendata_cleaned=greendata_cleaned.filter(greendata_cleaned.trip_duration>30)
greendata_cleaned=greendata_cleaned.filter(greendata_cleaned.trip_duration<3600)

In [0]:
yellowdata_cleaned.agg(F.percentile_approx("trip_duration", [0.05, 0.5, 0.95])).show(truncate=False)

In [0]:
yellowdata_cleaned=yellowdata_cleaned.filter(yellowdata_cleaned.trip_duration>30)
yellowdata_cleaned=yellowdata_cleaned.filter(yellowdata_cleaned.trip_duration<3600)

In [0]:
#cleaning based on trip distance#

In [0]:
greendata_cleaned.agg(F.percentile_approx("trip_distance", [0.05, 0.5, 0.95])).show(truncate=False)

In [0]:
greendata_cleaned=greendata_cleaned.filter(greendata_cleaned.trip_distance>1)
greendata_cleaned=greendata_cleaned.filter(greendata_cleaned.trip_distance<13)

In [0]:
yellowdata_cleaned.agg(F.percentile_approx("trip_distance", [0.05, 0.5, 0.95])).show(truncate=False)

In [0]:
yellowdata_cleaned=yellowdata_cleaned.filter(yellowdata_cleaned.trip_distance>1)
yellowdata_cleaned=yellowdata_cleaned.filter(yellowdata_cleaned.trip_distance<13)

In [0]:
#using an estimator to complete missing values

In [0]:
from pyspark.ml.feature import Imputer
imputer=Imputer(inputCols=["fare_amount", "extra","mta_tax","improvement_surcharge","tip_amount","tolls_amount","total_amount"], outputCols=["fare_amount", "extra","mta_tax","improvement_surcharge","tip_amount","tolls_amount","total_amount"]).setStrategy("median")
greendata_final=imputer.fit(greendata_cleaned).transform(greendata_cleaned)

In [0]:
from pyspark.ml.feature import Imputer
imputer=Imputer(inputCols=["fare_amount", "extra","mta_tax","improvement_surcharge","tip_amount","tolls_amount","total_amount"], outputCols=["fare_amount", "extra","mta_tax","improvement_surcharge","tip_amount","tolls_amount","total_amount"]).setStrategy("median")
yellowdata_final=imputer.fit(yellowdata_cleaned).transform(yellowdata_cleaned)

In [0]:
greendata_cleaned.fillna(0)
yellowdata_cleaned.fillna(0)

In [0]:
#column to see taxi types to answer future business questions

In [0]:
greendata_cleaned=greendata_cleaned.select('*',F.lit("green").alias("taxi_type"))
greendata_cleaned.show(1)


In [0]:
yellowdata_cleaned=yellowdata_cleaned.select('*',F.lit("yellow").alias("taxi_type"))
yellowdata_cleaned.show(1)

In [0]:
green_column=["VendorID","lpep_pickup_datetime","lpep_dropoff_datetime","trip_distance","PULocationID","DOLocationID","payment_type","passenger_count","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","trip_duration","taxi_type"]
yellow_column=["VendorID","tpep_pickup_datetime","tpep_dropoff_datetime","trip_distance","PULocationID","DOLocationID","payment_type","passenger_count","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","trip_duration","taxi_type"]

In [0]:
#Creating a final table

In [0]:
greendata_final=greendata_cleaned.select(green_column)
yellowdata_final=yellowdata_cleaned.select(yellow_column)

In [0]:
#Renaming certain variables

In [0]:
greendata_final=greendata_final.withColumnRenamed("lpep_pickup_time","pickup_time").withColumnRenamed("lpep_dropoff_time","dropoff_time")
yellowdata_final=yellowdata_final.withColumnRenamed("tpep_pickup_time","pickup_time").withColumnRenamed("tpep_dropoff_time","dropoff_time")

In [0]:
greendata_final.printSchema()
yellowdata_final.printSchema()

In [0]:
#Merging the datasets

In [0]:
data=greendata_final.union(yellowdata_final)

In [0]:
data.printSchema()

In [0]:
#Saving the data to Azure(when we tried to save inside DBFS I kept on getting an error

In [0]:
data.write.parquet('mnt/bdeat2/data2.parquet')

In [0]:
data_final=spark.read.option("header", True).parquet("mnt/bdeat2/data2*.parquet")

In [0]:
data_final=data_final.filter(data_final.lpep_pickup_datetime>F.to_timestamp(F.lit('01-01-2019 00:00:00.0000'), 'MM-dd-yyyy HH:mm:ss.SSSS'))
data_final=data_final.filter(data_final.lpep_pickup_datetime<F.to_timestamp(F.lit('05-01-2022 00:00:00.0000'), 'MM-dd-yyyy HH:mm:ss.SSSS'))

In [0]:
data_final.cache()

In [0]:
data_final.count()

In [0]:
data_final.createOrReplaceTempView("datacopy2")

In [0]:
sd2=spark.sql("SELECT *, YEAR(lpep_pickup_datetime) AS year, MONTH(lpep_pickup_datetime) AS month FROM datacopy2 ")

In [0]:
sd2.createOrReplaceTempView("sqldata2")

In [0]:
sd2.printSchema()

In [0]:
#Part 2

In [0]:
#Question 1.a

In [0]:
spark.sql("SELECT year, month, count(*) FROM sqldata2 WHERE year>2018 and year<2023 GROUP BY year, month ORDER BY year, month").show(42)

In [0]:
#Question 1.b

In [0]:
spark.sql("SELECT year, month, weekday, c FROM (SELECT year, month, c, weekday, ROW_NUMBER() OVER(PARTITION BY year, month  ORDER BY year, month, c DESC) as rn FROM (SELECT year, month, COUNT(*) AS c, WEEKDAY(lpep_pickup_datetime) AS weekday FROM sqldata2 WHERE year>2018 AND year<2023 GROUP BY year, month, weekday)ORDER BY year, month ) WHERE rn=1").show(50)

In [0]:
#Question 1 Part c

In [0]:
spark.sql("SELECT year, month, hour, c FROM (SELECT year, month, c, hour, ROW_NUMBER() OVER(PARTITION BY year, month  ORDER BY year, month, c DESC) as rn FROM (SELECT year, month, COUNT(*) AS c, HOUR(lpep_pickup_datetime) AS hour FROM sqldata2 WHERE year>2018 AND year<2023 GROUP BY year, month, hour)ORDER BY year, month ) WHERE rn=1").show(60)

In [0]:
#Question 1.d

In [0]:
spark.sql("SELECT year, month, AVG(passenger_count) FROM sqldata2 WHERE year>2018 AND year<2023 GROUP BY year, month ORDER BY year, month").show(40)

In [0]:
#question 1.e

In [0]:
spark.sql("SELECT year, month, SUM(total_amount)/count(*) AS avg_amount_per_trip FROM sqldata2 WHERE year>2018 AND year<2023 GROUP BY year, month ORDER BY year, month").show(40)

In [0]:
#question 1.f

In [0]:
spark.sql("SELECT year, month, SUM(total_amount)/SUM(passenger_count) AS avg_amt_per_passenger FROM sqldata2 WHERE year>2018 AND year<2023 GROUP BY year, month ORDER BY year, month").show(40)

In [0]:
#Question 2.a

In [0]:
spark.sql("SELECT taxi_type, round(AVG(trip_duration)/60::numeric,2) AS avg, round(MAX(trip_duration)/60::numeric,2) AS max, round(MIN(trip_duration)/60::numeric,2) as min FROM datacopy2 GROUP BY taxi_type").show(5)

In [0]:
spark.sql("SELECT taxi_type, round(AVG(trip_duration)/60::numeric,2) as median FROM (SELECT taxi_type, trip_duration, row_number() OVER(PARTITION BY taxi_type ORDER BY trip_duration ) AS rn, (SELECT COUNT(1) FROM datacopy2 WHERE taxi_type='green') as ct FROM datacopy2 ) WHERE rn BETWEEN ct/2.0 AND ct/2.0 +1 GROUP BY taxi_type").show(5)

In [0]:
spark.sql("SELECT taxi_type, round(AVG(trip_duration)/60::numeric,2) as median FROM (SELECT taxi_type, trip_duration, row_number() OVER(PARTITION BY taxi_type ORDER BY trip_duration ) AS rn, (SELECT COUNT(1) FROM datacopy2 WHERE taxi_type='yellow') as ct FROM datacopy2 ) WHERE rn BETWEEN ct/2.0 AND ct/2.0 +1 GROUP BY taxi_type").show(5)

In [0]:
#Question 2.b

In [0]:
spark.sql("SELECT taxi_type, AVG(trip_distance)*1.6 AS avg, MAX(trip_distance)*1.6 AS max, MIN(trip_distance)*1.6 as min FROM datacopy2 GROUP BY taxi_type").show(5)

In [0]:
spark.sql("SELECT taxi_type, AVG(trip_distance) as median FROM (SELECT taxi_type, trip_distance, row_number() OVER(PARTITION BY taxi_type ORDER BY trip_distance ) AS rn, (SELECT COUNT(1) FROM datacopy2 WHERE taxi_type='green') as ct FROM datacopy2 ) WHERE rn BETWEEN ct/2.0 AND ct/2.0 +1 GROUP BY taxi_type").show(5)

In [0]:
spark.sql("SELECT taxi_type, AVG(trip_distance)*1.6 as median FROM (SELECT taxi_type, trip_distance, row_number() OVER(PARTITION BY taxi_type ORDER BY trip_distance ) AS rn, (SELECT COUNT(1) FROM datacopy2) as ct FROM datacopy2) WHERE rn BETWEEN ct/2.0 AND ct/2.0 +1 GROUP BY taxi_type").show(5)

In [0]:
#Question 2.c

In [0]:
spark.sql("SELECT taxi_type, AVG(trip_distance)*1.6*3600/AVG(trip_duration) AS avg, MAX(trip_distance)*1.6*3600/MAX(trip_duration) AS max, MIN(trip_distance)*1.6*3600/MIN(trip_duration) as min FROM datacopy2 GROUP BY taxi_type").show(5)

In [0]:
spark.sql("SELECT taxi_type, AVG(trip_distance)*1.6*3600/AVG(trip_duration) as median FROM (SELECT taxi_type, trip_distance, trip_duration, row_number() OVER(PARTITION BY taxi_type ORDER BY trip_distance, trip_duration ) AS rn, (SELECT COUNT(1) FROM datacopy2) as ct FROM datacopy2) WHERE rn BETWEEN ct/2.0 AND ct/2.0 +1 GROUP BY taxi_type").show(5)

In [0]:
#Question 3

In [0]:
spark.sql("SELECT x.num*100/y.tot AS tips_received FROM (SELECT COUNT(*) AS num FROM datacopy2 WHERE tip_amount>0) x JOIN (SELECT COUNT(*) AS tot FROM datacopy2) y ON 1=1").show(5)

In [0]:
#Question 4

In [0]:
spark.sql("SELECT x.num*100/y.tot AS tips_received FROM (SELECT COUNT(*) AS num FROM datacopy2 WHERE tip_amount>10) x JOIN (SELECT COUNT(*) AS tot FROM datacopy2 WHERE tip_amount>0) y ON 1=1").show(5)

In [0]:
#Question 5

In [0]:
y=spark.sql("With table1 as(SELECT CASE WHEN trip_duration<=300 then 'Under 5 minutes' WHEN trip_duration > 300 and trip_duration <= 600 then 'Between 5 min - 10 min' WHEN trip_duration > 600 and trip_duration <= 1200 then 'Between 10 min - 20 min' WHEN trip_duration > 1200 and trip_duration <= 1800 then 'Between 20 min - 30 min' WHEN trip_duration > 1800 and trip_duration <= 3600 then 'Between 30 min -60 min' ELSE 'NA' End as trip_duration_bin, * FROM datacopy2 ) SELECT * FROM table1")

In [0]:
y.show(5)

In [0]:
y.createOrReplaceTempView("sqldata5")

In [0]:
spark.sql("SELECT trip_duration_bin, AVG(trip_distance)*1.6*3600/AVG(trip_duration) AS avg_speed FROM sqldata5 GROUP BY trip_duration_bin").show(10)

In [0]:
spark.sql("SELECT trip_duration_bin, AVG(trip_distance)*1.6/AVG(total_amount) AS avg_dist_dollar FROM sqldata5 GROUP BY trip_duration_bin").show(10)

In [0]:
#PART 3: Machine Learning: Random forest and Linear regression

In [0]:
data_final=data_final.select("VendorID","lpep_pickup_datetime","lpep_dropoff_datetime","trip_distance","PULocationID","DOLocationID","payment_type","passenger_count","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","trip_duration","taxi_type")

In [0]:
data_final.printSchema()

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
cat_cols=['taxi_type']
stages=[]
col_indexer=StringIndexer(inputCol="taxi_type", outputCol=f"taxi_type_ind")

In [0]:
data_final=col_indexer.fit(data_final).transform(data_final)

In [0]:
data_final.printSchema()

In [0]:
data_final.createOrReplaceTempView("datacopy3")

In [0]:
df1=spark.sql("SELECT *, YEAR(lpep_pickup_datetime) AS year, MONTH(lpep_pickup_datetime) AS month, WEEKDAY(lpep_pickup_datetime) AS weekday, HOUR(lpep_pickup_datetime) AS hour FROM datacopy3 ")

In [0]:
df1.createOrReplaceTempView("datacopy4")

In [0]:
df1=spark.sql("SELECT * FROM datacopy4 WHERE trip_distance IS NOT NULL AND PULocationID IS NOT NULL AND DOLocationID IS NOT NULL AND payment_type IS NOT NULL AND passenger_count IS NOT NULL AND extra IS NOT NULL AND mta_tax IS NOT NULL AND tip_amount IS NOT NULL AND tolls_amount IS NOT NULL AND improvement_surcharge IS NOT NULL AND total_amount IS NOT NULL AND trip_duration IS NOT NULL AND taxi_type_ind IS NOT NULL AND year IS NOT NULL AND month IS NOT NULL AND weekday IS NOT NULL AND hour IS NOT NULL")

In [0]:
df1.printSchema()

In [0]:
df1.show(2)

In [0]:
df1.fillna(value=0)
df1.dropna()

In [0]:
df1.count()

In [0]:
assembler=VectorAssembler(inputCols=["trip_distance","PULocationID","DOLocationID","payment_type","passenger_count","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","congestion_surcharge","trip_duration","taxi_type_ind","year","month","weekday","hour"], outputCol="features", handleInvalid="keep")

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_train=df1.filter(df1.lpep_pickup_datetime<F.to_timestamp(F.lit('04-01-2022 00:00:00.0000'), 'MM-dd-yyyy HH:mm:ss.SSSS'))
data_test=df1.filter(data_final.lpep_pickup_datetime>F.to_timestamp(F.lit('04-01-2022 00:00:00.0000'), 'MM-dd-yyyy HH:mm:ss.SSSS'))

In [0]:
data_train.count()
data_test.count()

In [0]:
rf_1=RandomForestRegressor(featuresCol='features', labelCol='total_amount')

In [0]:
pipeline=Pipeline(stages=[assembler, rf_1])

In [0]:
from pyspark.ml.tuning import ParamGridBuilder
import numpy as np
paramGrid = ParamGridBuilder().addGrid(rf_1.numTrees, [int(x) for x in np.linspace(start=10, stop=50, num=3)]).addGrid(rf_1.maxDepth, [int(x) for x in np.linspace(start=5, stop=15, num=3)]).build()

In [0]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
 
crossval=CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=RegressionEvaluator(), numFolds=3)

In [0]:
cvModel=crossval.fit(data_train)

In [0]:
predictions = cvModel.transform(Data_test)

In [0]:
import matplotlib.pyplot as plt
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

In [0]:
rfPred = cvModel.transform(df)
rfResult = rfPred.toPandas()
plt.plot(rfResult.label, rfResult.prediction, 'bo')
plt.xlabel('Price')
plt.ylabel('Prediction')
plt.suptitle("Model Performance RMSE: %f" % rmse)
plt.show()

In [0]:
print("RMSE: ", res.rootmeansquarederror)

In [0]:
#Linear Regression

In [0]:
pyspark.ml/regression import LinearRegression

In [0]:
lm=LinearRegression(featuresCol='features', labelCol='total_amount')

In [0]:
predictions2 = model2.transform(Data_test)

In [0]:
predictions2.show()

In [0]:
print("RMSE: ", res.rootmeansquarederror)